Importing Libraries

In [58]:
import json
import xmltodict

Converting Time given to seconds as easy to operate with

In [59]:
def convertTime(curTime):
    curTime = curTime[:8]
    parts = curTime.split(':')
    return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])

Reading the given .lwf file and making the list of all given leases by embedding it to the dictionaries

In [60]:
xml_file = open("data4.lwf")                                            #Parsing the .lwf file and reading attributes to create lease
data = xmltodict.parse(xml_file.read())["lease-workload"]
xml_file.close()

total_Leases=0
cloud_Nodes = int(data["site"]["nodes"]["node-set"]["@numnodes"])
strval = ""
leases = []
for request in data["lease-requests"]["lease-request"]:
    temp = {}
    temp["id"] = int(request["lease"]["@id"])
    temp["arrival"] = convertTime(request["@arrival"])
    temp["nodes"] = int(request["lease"]["nodes"]["node-set"]["@numnodes"])
    temp["start"] = convertTime(request["lease"]["start"]["exact"]["@time"])
    temp["duration"] = convertTime(request["lease"]["duration"]["@time"])
    temp["deadline"] = convertTime(request["lease"]["deadline"]["@time"])
    temp["slack"] = round((temp["deadline"] - temp["start"]) / temp["duration"], 2)
    if(temp["deadline"]>=temp["start"]):                    #checking if the lease values are valid
        leases.append(temp)
    total_Leases+=1

Sorting the leases based on arrival time

In [61]:
leases = sorted(leases, key=lambda d: d['arrival'])

Checking if the space with given time start and end time present in the given node or not

In [62]:
def check(node,start,end):                          #node space checking
    node.sort()
    for i in range(len(node)-1):
        if(node[i][1]<=start and end<=node[i+1][0]):
            return True
    return False


Finding the slot for given lease for the required number of nodes and if available appedning the given lease

In [63]:
def findSlot(req, resource):
    startTime = req["start"]
    deadline = req["deadline"]
    duration = req["duration"]
    required_nodes = req["nodes"]
    id=req["id"]
    found=False

    for i in range(len(resource)):
        for j in range(len(resource[i])-1):
            if(resource[i][j][1]>=startTime or startTime<=resource[i][j+1][0]):    #checkling if startime is grater than or equal to start time or less than next lease endtime  
                st=max(startTime,resource[i][j][1])                              #start time is max of start time and end time of previous end time
                if(st+duration<=resource[i][j+1][0] and st+duration<=deadline):      

                    available_nodes=[]                                  #available node for the scheduling for given start and duration

                    for x in range(len(resource)):
                        if check(resource[x],st,st+duration):                #checking the nodes for the given start and duration
                            available_nodes.append(x)

                    if len(available_nodes)>=required_nodes: 
                        
                        tmp=[st,st+duration,id]
                        for x in range(required_nodes):
                            resource[available_nodes[x]].append(tmp)
                            resource[available_nodes[x]].sort()
                        found=True
                        break
        
        if found:
            return True
    return False             

            

If the node space could not found then trying the rescheduling by sorting on the slack values

In [64]:
def reschedule(req,resource):

    duplicateResource = []                      #Creating dummy resource for rescheduling
    count=0
    st=[0,0,0] 
    en=[1e9,1e9,1e9]

    for i in range(cloud_Nodes):
        tmp=[]
        tmp.append(st)
        tmp.append(en)
        duplicateResource.append(tmp)

    nonScheduleLeases=[]
    st=req["start"]

    for i in range(len(resource)):
        for j in range(1,len(resource[i])-1):
           if(st<=resource[i][j][0]):
            nonScheduleLeases.append(resource[i][j][2])
           else:
            duplicateResource[i].append(resource[i][j])

    s=set(nonScheduleLeases)
    nonScheduleLeases=list(s)
    nonScheduleLeasesDict=[]
    nonScheduleLeasesDict.append(req)

    for i in range(len(leases)):
        for j in range(len(nonScheduleLeases)):
            if(leases[i]["id"]==nonScheduleLeases[j]):
                nonScheduleLeasesDict.append(leases[i])

    nonScheduleLeasesDict = sorted(nonScheduleLeasesDict, key=lambda d: d['slack'])
    for i in range(len(nonScheduleLeasesDict)):
        if not findSlot(nonScheduleLeasesDict[i],duplicateResource):                #If any one of the lease not schedule then reeturn original resource
            return resource,True,nonScheduleLeases

    return duplicateResource,False,nonScheduleLeases                                                  #If able to reschedule all the leases return the duplicate resource


Creating the node spaces and and calling the findslot function and if could not schedule trying to reschedule

In [65]:
resource = []
count=0
st=[0,0,0] 
en=[1e9,1e9,1e9]

rescheduling_insisted=0
cant_schedule=0

for i in range(cloud_Nodes):
    tmp=[st,en]
    resource.append(tmp)


for i in range(len(leases)):
     if not findSlot(leases[i],resource):
        rescheduling_insisted+=1
        resource,didnt,leases_rescheduled=reschedule(leases[i],resource)
        cant_schedule+=didnt
        if not didnt:
            print("Leases that rescheduled for the lease id "+str(leases[i]["id"])+ " are with ids "+str(leases_rescheduled))  




#calculating the utilization for all the nodes


used_time=0
all_resource_time=0

for x in resource:
    used1=0
    for i in range(len(x)-1): 
        used_time+=(x[i][1]-x[i][0])
    all_resource_time+=x[len(x)-2][1]

utilization=(used_time/all_resource_time)
print("Total Leases:", total_Leases)
print("Total Leases Scheduled:", total_Leases - (cant_schedule+total_Leases-len(leases)))
print("Total non-scheduled leases: ",(cant_schedule+total_Leases-len(leases)))
print("Total leases that insisted rescheduling: ",rescheduling_insisted)
print("Scheduled Percentage:", (round(((total_Leases - (cant_schedule+total_Leases-len(leases)))/ total_Leases)*100,2)))
print("Utilization of reesources:-" +str(utilization)+"\n")


Leases that rescheduled for the lease id 31 are with ids [7, 8, 9, 13, 14, 15, 16, 18, 20]
Leases that rescheduled for the lease id 34 are with ids [33, 9, 13, 14, 15, 16, 18, 20]
Leases that rescheduled for the lease id 36 are with ids [13, 14, 15, 16, 18, 20]
Total Leases: 43
Total Leases Scheduled: 24
Total non-scheduled leases:  19
Total leases that insisted rescheduling:  22
Scheduled Percentage: 55.81
Utilization of reesources:-0.3719298245614035

